# Build the Neural Network

Neural networks comprise of layers/modules that perform operations on data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to build your neural network. Every module in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset

In [1]:
import os
import torch
# from torch import nn
# from torch.utils.data import DataLoader
# from torchvision import datasets, transforms

## Get Device for Training

We want to be able to train our model on an [accelerator](https://pytorch.org/docs/stable/torch.html#accelerators) such as CUDA, MPS, MTIA, or XPU. If the current accelerator is available, we will use it. Otherwise, we use the CPU.

In [2]:
try:
    device = torch.accelerator.current_accelerator.type if torch.accelerator.is_available() else "cpu"
    print(f"Using {device} device")
except:
    device = 'cpu'
    print('The CPU-only version of PyTorch is installed -- no torch.accelerator module available.')

The CPU-only version of PyTorch is installed -- no torch.accelerator module available.


## Define the Class

We define our neural network by subclassing `nn.Module` and initialize the neural network layers in `__init__`. Every `nn.Module` subclass implements the operations on input data in the `forward` method.

In [3]:
class NeuralNetwork(torch.nn.Module):
    '''
    This is a 2-hidden-layer multilayer perceptron (MLP) with 512 units per
    hidden layer and ReLU activations. It maps a 28x28 grayscale image to a
    10-class classification output. Note: 28**2 = 784
    Forward pass flow:
        [N,1,28,28] -> [N,784] -> [N,512] -> [N,512] -> [N,10]
    '''
    def __init__(self):
        super().__init__()
        self.flatten = torch.nn.Flatten()
        self.linear_relu_stack = torch.nn.Sequential(  # ordered container, runs layers in sequence.
            torch.nn.Linear(28*28, 512),  # Layer 1: fully connected, projects flattened image -> 512 features
            torch.nn.ReLU(),              # Activation 1: applies ReLU element-wise (max(0,x)), introducing nonlinearities
            torch.nn.Linear(512, 512),    # Layer 2: hidden layer, keeps feature dimension 512
            torch.nn.ReLU(),              # Activation 2: applies ReLU element-wise (max(0,x)), introducing nonlinearities
            torch.nn.Linear(512, 10)      # Layer 3: output layer, 10 units = 1 unit per FashionMNIST class
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`, and print its structure.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model's `forward`, along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866). Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of eah output. We get the prediction probabilities by passing it through an instance of `nn.Softmax` module.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = torch.nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5])


## Model Layers

Let's break down the layers in the fashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### `nn.Flatten`

We initialize the [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (the minibatch dimension (at dim=0) is maintained).

In [7]:
flatten = torch.nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### `nn.Linear`

The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) is a module that applies a linear transformation on the input using its stored weights and biases.

In [8]:
layer1 = torch.nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### `nn.ReLU`

Non-linear activations are what create the complex mappings between the model's inputs and outputs. They are applies after linear transformations to introduce *nonlinearity*, helping neural networks to approximate more complex functions.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our linear layers, but there's other activations to introduce non-linearity in your model.

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = torch.nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3219,  0.1632,  0.0229,  0.0318,  0.2201,  0.2375,  0.2808, -0.0542,
          0.2431,  0.5279,  0.1065,  0.0989,  0.0671, -0.2326, -0.0256,  0.4565,
          0.1842, -0.0632, -0.9052,  0.4593],
        [ 0.2207, -0.1200,  0.2723,  0.1421,  0.4299,  0.1876,  0.5424, -0.4376,
          0.3253,  0.1706, -0.0618,  0.2151,  0.0845, -0.4501,  0.2252,  0.2839,
          0.1492,  0.1834, -0.8131,  0.3164],
        [ 0.1355,  0.0902,  0.1765,  0.5433,  0.6909,  0.3342,  0.3858, -0.0918,
          0.4885,  0.1792,  0.1420,  0.8687,  0.0061, -0.3581, -0.1100, -0.1651,
          0.1445,  0.2705, -0.5637,  0.6648]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3219, 0.1632, 0.0229, 0.0318, 0.2201, 0.2375, 0.2808, 0.0000, 0.2431,
         0.5279, 0.1065, 0.0989, 0.0671, 0.0000, 0.0000, 0.4565, 0.1842, 0.0000,
         0.0000, 0.4593],
        [0.2207, 0.0000, 0.2723, 0.1421, 0.4299, 0.1876, 0.5424, 0.0000, 0.3253,
         0.1706, 0.0000, 0.2151, 0.0845, 0.0000, 0.22

### `nn.Sequential`

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like `seq_modules`.

In [10]:
seq_modules = torch.nn.Sequential(
    flatten,
    layer1,
    torch.nn.ReLU(),
    torch.nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### `nn.Softmax`

The last linear layer of the neural network returns *logits* - raw values in \[-infty, infty\] - which are passed to the [nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. The logits are scaled to values \[0, 1\] representing the model's predicted probabilities for each class. `dim` parameter indicated the dimension along which the values must sum to 1.

In [11]:
softmax = torch.nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model Parameters

Many layers inside a neural network are *parameterized*, i.e., have associated weights and biases that are optimized during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model's `parameters()` or `named_parameters` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0118, -0.0175, -0.0148,  ..., -0.0006,  0.0309, -0.0051],
        [-0.0314, -0.0119,  0.0152,  ..., -0.0070, -0.0276, -0.0191]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0098, -0.0185], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0015,  0.0106, -0.0271,  ...,  0.0327,  0.0035,  0.0215],
        [-0.0011,  0.0161, -0.0354,  ...,  0.0017, -0.0257,  0.0049]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 